In [89]:
import os
import json
import google.generativeai as genai
from tavily import TavilyClient
from dotenv import load_dotenv
from IPython.display import display, Markdown, JSON
import google.generativeai as genai
import urllib.parse
import json
import os
import requests
import base64
from dotenv import load_dotenv



In [90]:
# Load environment variables
load_dotenv()

# --- CONFIGURATION ---
# Replace these with your actual keys or ensure they are in your .env file
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") 
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

if not GEMINI_API_KEY or not TAVILY_API_KEY:
    print("⚠️ WARNING: Please set GEMINI_API_KEY and TAVILY_API_KEY in your environment or this cell.")

# Configure Clients
genai.configure(api_key=GEMINI_API_KEY)
#TODO change model
model = genai.GenerativeModel('gemini-2.5-flash') 
tavily = TavilyClient(api_key=TAVILY_API_KEY)

In [91]:
### --- BOLAGSVERKET AUTHENTICATION & API CAllS ---

# Load environment variables from a .env file
load_dotenv()

CLIENT_ID = os.getenv("BOLAGSVERKET_CLIENT_ID", "YOUR_CLIENT_ID")
CLIENT_SECRET = os.getenv("BOLAGSVERKET_CLIENT_SECRET", "YOUR_CLIENT_SECRET")

# Endpoints documented by Bolagsverket
TOKEN_URL = "https://portal.api.bolagsverket.se/oauth2/token"
# Base URL found in documentation for Värdefulla datamängder
API_BASE_URL = "https://gw.api.bolagsverket.se/vardefulla-datamangder/v1"

def get_access_token():
    """Authenticates with Bolagsverket and returns an access token."""
    
    # Encode client_id:client_secret in base64 for Basic Auth header
    creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
    creds_b64 = base64.b64encode(creds.encode("utf-8")).decode("utf-8")

    headers = {
        "Authorization": f"Basic {creds_b64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "client_credentials",
        # needed for read / ping access 
        "scope": "vardefulla-datamangder:read vardefulla-datamangder:ping" 
    }

    try:
        response = requests.post(TOKEN_URL, headers=headers, data=data)
        response.raise_for_status()
        token_data = response.json()
        return token_data["access_token"]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching token: {e}")
        if response.content:
            print(f"Details: {response.content}")
        return None


def search_company(org_number, token):
    """
    Fetches company information using the provided organization number and access token.
    Args:
        org_number (str or int): The 10-digit tax number of the organization. If the input 
            is not 10 digits, it will be zero-padded to ensure the correct format.
        token (str): The access token required for authentication.
    Returns:
        dict: A dictionary containing the company information retrieved from the API, 
            if the request is successful.
        None: If the request fails, returns None and logs the error details.
    Notes:
        - The organization number should be provided without any dashes (e.g., "5560160680").
        - Ensure that the `API_BASE_URL` variable is defined and points to the correct API endpoint.
        - The function prints detailed error messages to help debug issues with the API response.
    """
    
    url = f"{API_BASE_URL}/organisationer"
    
    # ---------------------------------------------------------
    # Use tax id of comapny -> without - 
    # ---------------------------------------------------------
    payload = {
        "identitetsbeteckning": org_number
    }

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching data: {e}")
        # Print detailed error to debug schema issues
        print(f"Response: {response.text}")
        return None
    


In [ ]:
def generate_queries(company_name: str, org_id: str = None) -> list[str]:
    """
    Ask Gemini to generate optimized search queries based on the entity name.
    """
    input_data = f"""
    <user>
        <input>
            <company_info>
                <name>{company_name}</name>
                <company_id>{org_id or "Unknown"}</company_id>
            </company_info>
        </input>
    </user>
    """
    
    with open("../src/app/services/prompts/query_internet.xml", "r") as file:
        QUERY_INTERNET_PROMPT = file.read()
    full_prompt = QUERY_INTERNET_PROMPT + input_data
    
    response = model.generate_content(
        full_prompt, 
        generation_config={"response_mime_type": "application/json"}
    )
    
    try:
        data = json.loads(response.text)
        # Extract just the query strings from the structured objects
        queries = [q["query"] for q in data.get("queries", [])]
        # Deduplicate
        return list(set(queries))
    except Exception as e:
        print(f"Error parsing Gemini response: {e}")
        return [f"{company_name} official website", f"{company_name} Sweden annual report"]

def perform_search(queries: list[str], max_results: int = 5):
    """
    Step 2: Execute search using Tavily API.
    """
    aggregated_context = []
    
    print(f"🔎 Executing {len(queries)} search queries...")
    
    # To save tokens/API credits, we might limit queries here
    # TODO test values 
    for query in queries[:3]: # Limit to top 3 queries for this demo
        try:
            print(f"   -> Searching: '{query}'")
            # Tavily 'search' returns structured results with content
            response = tavily.search(query=query, search_depth="advanced", max_results=5)
            
            for result in response.get("results", []):
                aggregated_context.append(f"Source: {result['url']}\nContent: {result['content']}\n---")
        except Exception as e:
            print(f"   x Error searching '{query}': {e}")
            
    return "\n".join(aggregated_context)

def extract_org_id_from_context(search_context: str, company_name: str) -> str:
    """
    Scans the provided search context (text) to find a Swedish Organization Number.
    Does NOT perform a new web search.
    """
    print(f"🕵️‍♀️ analyzing text to extract Org ID for: {company_name}...")

    if not search_context:
        print("   ⚠️ No context provided to analyze.")
        return None

    with open("../src/app/services/prompts/query_internet.xml", "r") as file:
        ID_PROMPT = file.read()
    full_prompt = ID_PROMPT + search_context
    
    try:
        response = model.generate_content(extraction_prompt)
        org_id = response.text.strip().replace('"', '').replace("'", "") # Clean up quotes
        
        if "None" in org_id or len(org_id) < 10:
            print("   ⚠️ Could not confidently extract ID from context.")
            return None
            
        print(f"   ✅ Found Org ID: {org_id}")
        return org_id
        
    except Exception as e:
        print(f"   x Extraction failed: {e}")
        return None

def structure_data(company_name: str, search_context: str):
    """
    Step 3: Structure the gathered raw text into the Northern Lights JSON schema.
    """
    input_data = f"""
    <user>
        <input>
            <source_data>
                <bolagsverket>
                    Legal Name: {company_name}
                    Registered: Sweden
                </bolagsverket>
                <web_search>
                    {search_context}
                </web_search>
            </source_data>
            
            <entity_context>
                <entity_name>{company_name}</entity_name>
                <entity_type>company</entity_type>
            </entity_context>
        </input>
    </user>
    """
    
    with open("../src/app/services/prompts/structure_data.xml", "r") as file:
        STRUCTURE_DATA_PROMPT = file.read()
    full_prompt = STRUCTURE_DATA_PROMPT + input_data
    print(full_prompt)
    response = model.generate_content(
        full_prompt, 
        generation_config={"response_mime_type": "application/json"}
    )
    
    try:
        return json.loads(response.text)
    except Exception as e:
        print(f"Error structuring data: {e}")
        return {"error": response.text}

In [93]:
# --- RUN THE PIPELINE ---

# 1. Define Target
COMPANY_NAME = "Ericsson"
print(f"Starting Scraper for: {COMPANY_NAME}\n")

# 2. Generate Queries
queries = generate_queries(COMPANY_NAME)
print("✅ Generated Queries:")
print(json.dumps(queries[:2], indent=2))

# 3. Scrape Web
if queries:
    search_context = perform_search(queries)
    print(f"\n✅ Retrieved {len(search_context)} characters of context.")
else:
    search_context = ""
    print("❌ No queries generated.")
    

if search_context:
    TARGET_ORG_ID = extract_org_id_from_context(search_context, COMPANY_NAME)
else:
    print("❌ Cant find TARGET_ORG_ID due to lacking context.")

# CAll API 
# Format the organization ID to remove the dash
if TARGET_ORG_ID:
    TARGET_ORG_ID = TARGET_ORG_ID.replace("-", "")
print(f"Extracted Organization ID: {TARGET_ORG_ID}")
print("1️⃣  Authenticating...")
# Call the Bolagsverket API to get more informaiton on the company 
token = get_access_token()
if token:
    print("✅ Access Token received!")

    
    print(f"2️⃣  Searching for company: {TARGET_ORG_ID}...")
    boglagsverket_api_data = search_company(TARGET_ORG_ID, token)
    
    if boglagsverket_api_data:
        print("✅ boglagsverket_api_data received:")
        # Print the first few lines of the JSON response for better readability
        print(json.dumps(boglagsverket_api_data, indent=2)[:500])
else:
    print("🛑 Fail to retrieve Bolagsverket token.")


# 4. Structure Data
if search_context:
    print("\n🧠 Structuring data with Gemini...")
    structured_data = structure_data(COMPANY_NAME, search_context)
    
    print("\n✨ FINAL JSON OUTPUT:")
    display(JSON(structured_data))
else:
    print("❌ Skipping structuring due to lack of context.")

Starting Scraper for: Ericsson

✅ Generated Queries:
[
  "Ericsson Wikipedia",
  "Ericsson officiell hemsida"
]
🔎 Executing 36 search queries...
   -> Searching: 'Ericsson Wikipedia'
   -> Searching: 'Ericsson officiell hemsida'
   -> Searching: 'Ericsson vision purpose'

✅ Retrieved 17908 characters of context.
🕵️‍♀️ analyzing text to extract Org ID for: Ericsson...
   ⚠️ Could not confidently extract ID from context.
Extracted Organization ID: None
1️⃣  Authenticating...
✅ Access Token received!
2️⃣  Searching for company: None...
❌ Error fetching data: 400 Client Error:  for url: https://gw.api.bolagsverket.se/vardefulla-datamangder/v1/organisationer
Response: {"timestamp":"2025-12-05T23:55:39.289+00:00","status":400,"error":"Bad Request","message":"Validation failed for object='organisationerBegaran'. Error count: 1","path":"/v1/organisationer"}

🧠 Structuring data with Gemini...
<system>
    <mission>
        You are a data structuring specialist for Northern Lights, a Nordic corp

<IPython.core.display.JSON object>

In [94]:
org_id = structured_data.get('organization_id')
print(f"Extracted Organization ID: {org_id}")
print("1️⃣  Authenticating...")
# Call the Bolagsverket API to get more informaiton on the company 
token = get_access_token()
if token:
    print("✅ Access Token received!")
    
    # Example: Search for Bolagsverket's own org number (202100-5489)
    # Remove hyphen for the API: 2021005489
    test_org_number = 9697802230
    
    print(f"2️⃣  Searching for company: {test_org_number}...")
    boglagsverket_api_data = search_company(test_org_number, token)
    
    if boglagsverket_api_data:
        print("✅ Data received:")
        print(boglagsverket_api_data)
else:
    print("🛑 Could not proceed without token.")

Extracted Organization ID: None
1️⃣  Authenticating...
✅ Access Token received!
2️⃣  Searching for company: 9697802230...
✅ Data received:
{'organisationer': [{'avregistreradOrganisation': {'avregistreringsdatum': '2018-01-31', 'dataproducent': 'Bolagsverket', 'fel': None}, 'avregistreringsorsak': {'dataproducent': 'Bolagsverket', 'fel': None, 'klartext': 'Anmälan om att verksamheten har upphört', 'kod': 'VERKUPP'}, 'juridiskForm': {'dataproducent': 'SCB', 'fel': {'felBeskrivning': 'Den efterfrågade informationen gick inte att hitta.', 'typ': 'ORGANISATION_FINNS_EJ'}, 'klartext': None, 'kod': None}, 'namnskyddslopnummer': None, 'naringsgrenOrganisation': {'dataproducent': 'SCB', 'fel': {'felBeskrivning': 'Den efterfrågade informationen gick inte att hitta.', 'typ': 'ORGANISATION_FINNS_EJ'}, 'sni': []}, 'organisationsdatum': {'dataproducent': 'Bolagsverket', 'fel': None, 'infortHosScb': None, 'registreringsdatum': '2016-07-07'}, 'organisationsform': {'dataproducent': 'Bolagsverket', 'fe